## Machine-Learning Atomic Charges for Efficient Molecular Dynamics Simulations

### Motivation
Partitioning the electron density of a molecule (or collection of molecules)
is of fundamental importance in simulating molecular motion using molecular
dynamics. Traditionally, partitioning the electron density requires a 
relatively expensive wavefunction or density functional theory (DFT) calculation,
methods that formally scale as at least $\mathcal{O}(N^3)$. By contrast, the underlying
molecular dynamics scales approximately as $\mathcal{O}(N)$, making the DFT the
rate-limiting step.


Here, we construct an empirical model of atomic charge partitioning with
asymptotic $\mathcal{O}(N^2)$ cost, reducing the formal scaling of the entire method
by $\mathcal{O}(N)$.


To accomplish this, we map an atom in its environment to a corresponding
scalar, its atomic charge. The atomic charge label is computed by the minimal
basis iterative stockholder (MBIS) algorithm performed on a second-order
perturbative wavefunction from the simplest truncation of symmetry-adapted
perturbation theory (SAPT0).

i.e. We seek the function that maps the atomic environment ($\vec{X}$) to its
corresponding atomic charge ($y$):

$\vec{X} \leftarrow y$

Here, choosing $\vec{X}$ (Coulomb matrices, symmetry functions, graph convolutions),
and an appropriate functional form (KRR, k-NN, MLP) are critical in achieving
a sufficiently fast and physically meaningful mapping.

In [1]:
import numpy as np
#import tensorflow as tf
import matplotlib.pyplot as plt
from scipy.spatial import distance_matrix
from sklearn.model_selection import cross_validate
from sklearn.neural_network import MLPRegressor

Here are simple imports for what we'll need. For more complex model architectures, Google's neural network library TensorFlow may be useful. We find excellent results with the simple MLP implementation in scikit-learn.


### Featurization

Now, we'll define a couple options for $\vec{X}$, our "atomic environment vectors" -- the (local) Coulomb matrix and symmetry functions:

The Coulomb matrix is defined as
$$C_{ij} =
      \left\{ \begin{array}{ll}
          \frac{Z_i Z_j}{r_{ij}}& i \neq j \\
          Z_i^2 & i = j
      \end{array} \right.$$
The local version of the Coulomb matrix, used for efficiency and constant vector sizes, only considers the nearest $n$ $j$ in Euclidian distance. The matrix is flattened and fed as input to any statistical algorithm.

The radial symmetry function is defined as 
$$G_{i}^{rad}(\mu, \eta, Z) = \sum_{j \ne i}e^{-\eta(r_{ij}-\mu)^2}f_{c}(r_{ij})\delta_{ZZ_{j}},$$ with cutoff function $f_c(r_{ij})$ to enforce locality, $$f_{c}(r_{ij})=
      \left\{ \begin{array}{ll}
          \frac{1}{2}(\cos(\frac{r_{ij}}{r_c}) + 1) & r_{ij}\leq r_c \\
          0 & r_{ij} > r_c
      \end{array} \right. ,$$
and $\delta_{ZZ_{j}}$ is the Kronecker delta, enforcing different "channels" for different nearby nuclei.


Both of these have options, which can be seen as hyperparameters. For the local Coulomb matrix, $n$ decides how many atomic neighbors are considered. The symmetry functions, which are simply a set of radial Gaussian shells, have an $\eta$ parameter, determining the width of each Gaussian, $\mu$, determining the offset of each gaussian from the origin atom, and $r_c$, the cutoff distance. In practice, $r_c$ rarely exceeds $5$ angstroms, $\mu$ is evenly spaced between a small number ($0.5$ angstroms, or so) and $r_c$, and $\eta$ is treated as a true hyperparameter.

In [2]:
def get_local_coulomb(R, Z, n=5):
    """
    Generates local Coulomb matrix for set of molecules
    Args: Unpadded list of cartesian coords, atom nums
        optional: number of neighbors to consider default=5
    Returns: Unrolled list of local atomic coulomb matrices
    """
    C = []
    dists = []
    for i, coords in enumerate(R):
        dists = distance_matrix(coords, coords)
        for j in range(dists.shape[0]):
            inds = dists[j].argsort()[:n]
            R_local = np.zeros((n,n))
            ZZ_local = np.zeros((n,n))
            for k in range(len(inds)):
                for l in range(len(inds)):
                    if k != l:
                        R_local[k][l] = dists[inds[k],inds[l]]
                    elif k == l:
                        R_local[k][l] = 1.0 #fixes diagonal inf
                    ZZ_local[k][l] = Z[i][inds[k]] * Z[i][inds[l]]
            C.append(ZZ_local / R_local)
    C = np.array(C)
    return np.reshape(C, (C.shape[0], C.shape[1]**2))

def get_symmetry_functions(R, Z, eta=2., num = 20, r_c = 5.):
    """
    Generates weighted radial symmetry functions for each atom in a set of
    molecules a la Behler, Parrinello, Gastegger, Marquetand.
    Args: Unpadded list of Cartesian coords, atom nums
        optional:
         eta      : hyperparameter, corresponding to Gaussian widths
         num : hyperparameter, corresponding to number of Gaussians
         r_c      : hyperparameter, corresponding to cutoff radius
    Returns: Unrolled list of atomic radial symmetry functions
    """
    Z_dict = {1.0 : 0,
              6.0 : 1,
              7.0 : 2,
              8.0 : 3,
              9.0 : 4,
              16.0: 5,}

    G = []
    mu = np.linspace(0.8, r_c, num)
    for i, coords in enumerate(R):
        dists = distance_matrix(coords, coords)
        for j in range(dists.shape[0]):
            atom_G = np.zeros(len(mu)*len(Z_dict))
            for k in range(dists.shape[0]):
                if j != k:
                    if dists[j][k] < r_c:
                        cutoff = 0.5 * (np.cos(dists[j][k] / r_c) + 1)
                    else: cutoff = 0.0
                    exp_term = np.exp(-eta * dists[j][k] - mu) ** 2
                    ind = Z_dict[Z[i][k]] * len(mu)
                    atom_G[ind:ind+len(mu)] += exp_term * cutoff
            G.append(atom_G)
    return np.array(G)

These implementations return a flattened set of vectors, each vector describing an atom's environment. Therefore, the shape of the Coulomb matrix function's return is (number_of_atoms, n^2). The shape of the symmetry function return is (number_of_atoms, number_of_elements * num), where num is the hyperparameter controlling the number of radial Gaussians.

### Loading in data

In [ ]:
def reconstitute(RAp, RBp, ZAp, ZBp, QAp, QBp):
    """
    Args: padded individual monomer coordinates, atom nums, and charges
    Returns: unpadded, concatenated coords, atom nums, and charges
    """
    R = []
    Z = []
    Q = []
    for i in range(len(RAp)):
        R_mol = []
        Z_mol = []
        Q_mol = []
        for j in range(len(RAp[i])):
            if ZAp[i][j] != 0:
                R_mol.append(RAp[i][j])
                Z_mol.append(ZAp[i][j])
                Q_mol.append(QAp[i][j])
        for j in range(len(RBp[i])):
            if ZBp[i][j] != 0:
                R_mol.append(RBp[i][j])
                Z_mol.append(ZBp[i][j])
                Q_mol.append(QBp[i][j])
        R.append(np.array(R_mol))
        Z.append(np.array(Z_mol))
        Q.append(np.array(Q_mol))
    return R, Z, Q

# load in system names, electrons, atomic numbers, and coordinates
names = np.load("bms_data/names_train.npy")
e_A = np.load("bms_data/Q_A_train.npy")
e_B = np.load("bms_data/Q_B_train.npy")
Z_A = np.load("bms_data/Z_A_train.npy")
Z_B = np.load("bms_data/Z_B_train.npy")
R_A = np.load("bms_data/R_A_train.npy")
R_B = np.load("bms_data/R_B_train.npy")
    
Q_A = Z_A - e_A
Q_B = Z_B - e_B
    
# these are split by monomer and padded, reconstituting total system w/o pad
R, Z, Q = reconstitute(R_A, R_B, Z_A, Z_B, Q_A, Q_B)

These data are molecular dimers whose chemistry is relevant in drug discovery. Specifically, the training configurations have been artificially sampled based on the intuition of a team of medicinal chemists, not drawn from any experimental study. Being loaded and parsed are
1. R : Cartesian coordinates of each atom in each system
2. Z : Atomic numbers of each atom in each system
3. Q : Atomic charges (y), which we seek to predict

Also included in the bms_data directory are "*_val.npy" files, which will be used as a final validation set. These are sensible choices of validation data because these molecular dimers are drawn from experimental crystallographic data, so are by construction physically reasonable configurations that medicinal chemists might be interested in.

Good performance on the crystallographic validation set would indicate that the training and testing sets, both drawn from artificial configurations, are sufficient to model the underlying physics of electron distributions.

### Generating features and training models

Below is code that generates local Coulomb matrices and radial symmetry functions, given the input R and Z:

In [ ]:
## generate atomic environment descriptors

#X = get_local_coulomb(R, Z, n=5)
X = get_symmetry_functions(R, Z, eta=1., num=15)

##The following feature represents the null hypothesis, where atoms
## are defined only by their atomic number. As you would expect,
## this doesn't work very well.
#C = np.expand_dims(np.hstack(Z), -1)
##

y = np.hstack(Q)

mlp = MLPRegressor(hidden_layer_sizes=(50,50), early_stopping=True, n_iter_no_change=10)
scoring = ('r2', 'neg_mean_absolute_error')
cv_results = cross_validate(mlp, X, y, cv=3, scoring=scoring, return_train_score=True)
    
print(cv_results)

This preliminary effort makes available many optimizations for improving fast charge predictions:

1. Exploring descriptor hyperparameters


    a. Local Coulomb matrices depend on n, a natural number referring to the number of atomic neighbors to consider.   
    b. Symmetry functions depend on $\eta$, num, and $r_c$, determining Gaussian widths, number of Gaussians, and cutoff radius, respectively.
    
    
2. Exploring model types


    a. KRR is accurate with little data but is potentially expensive with the quantity of data we have
    b. Neural networks are fast at inference-time but have many hyperparameters and can be cumbersome to train
    c. k-NN is potentially accurate but has high memory cost and non-smooth inferences w.r.t. molecular motion
    
    
3. Exploring model hyperparameters -- each of the above have several hyperparameters that will |affect performance